In [1]:
!pip uninstall scikit-learn imbalanced-learn -y

# Then install compatible versions
!pip install scikit-learn==1.4.0 imbalanced-learn==0.11.0



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif, RFECV, RFE
from sklearn.decomposition import PCA
from sklearn.ensemble import  RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import label_binarize
from sklearn import metrics
import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN,RandomOverSampler


Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
Found existing installation: imbalanced-learn 0.13.0
Uninstalling imbalanced-learn-0.13.0:
  Successfully uninstalled imbalanced-learn-0.13.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 11.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.


In [2]:
from collections import Counter

In [3]:
df=pd.read_csv("/kaggle/input/sleep-health-and-lifestyle-dataset/Sleep_health_and_lifestyle_dataset.csv")

df['Sleep Disorder'] = df['Sleep Disorder'].fillna("okay")
df.head()

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,okay
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,okay
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,okay
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea


In [4]:
df=pd.read_csv("/kaggle/input/sleep-health-and-lifestyle-dataset/Sleep_health_and_lifestyle_dataset.csv")

df['Sleep Disorder'] = df['Sleep Disorder'].fillna("okay")



import pandas as pd

df=df.drop(['Person ID'],axis=1)
# One-hot encode specific columns
df = pd.get_dummies(df, columns=['Gender', 'Occupation', 'BMI Category','Blood Pressure'], drop_first=False)




from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['Sleep Disorder'] = le.fit_transform(df['Sleep Disorder'])

from sklearn.model_selection import train_test_split
target_col='Sleep Disorder'
X = df.drop(columns=[target_col])
y = df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [5]:
print(X_train.shape, X_test.shape)

(280, 49) (94, 49)


In [6]:
ML_Model = []
ML_Config = []
accuracy = []
f1_score = []
recall = []
precision = []
auc_roc = []  # Adding a holder for AUC-ROC


def storeResults(model, config, a, b, c, d, e):
    """
    Store model performance results
    
    Parameters:
    model: Name of the ML model
    config: Configuration name (preprocessing steps applied)
    a: Accuracy score
    b: F1 score
    c: Recall score
    d: Precision score
    e: AUC-ROC score
    """
    ML_Model.append(model)
    ML_Config.append(config)
    accuracy.append(round(a, 6))
    f1_score.append(round(b, 6))
    recall.append(round(c, 6))
    precision.append(round(d, 6))
    auc_roc.append(round(e, 6))


configurations = []
configurations.append(('Original Data', X_train, X_test, y_train))

# Step 1: Normalize the data
scaler = MinMaxScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)
configurations.append(('Normalized Data', X_train_normalized, X_test_normalized, y_train))

# === Step 1.5 (NEW): Oversampling variants on the normalized training set ===
# We oversample ONLY the training fold; test fold stays untouched for fair evaluation.
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss
from imblearn.under_sampling import TomekLinks

from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE, SVMSMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss, TomekLinks
from imblearn.combine import SMOTETomek, SMOTEENN

samplers = [
    # Over-sampling techniques
    ('Normalized + RandomOverSampler', RandomOverSampler(
        sampling_strategy='auto',
        random_state=42
    )),
    ('Normalized + SMOTE', SMOTE(
        sampling_strategy='auto',
        k_neighbors=5,
        random_state=42,
        n_jobs=-1
    )),
    ('Normalized + Borderline-SMOTE', BorderlineSMOTE(
        kind='borderline-1',
        k_neighbors=5,
        m_neighbors=20,
        sampling_strategy='auto',
        random_state=42,
        n_jobs=-1
    )),
    ('Normalized + SVMSMOTE', SVMSMOTE(
        k_neighbors=5,
        m_neighbors=20,
        sampling_strategy='auto',
        random_state=42,
        n_jobs=-1
    )),
    ('Normalized + ADASYN', ADASYN(
        sampling_strategy='auto',
        n_neighbors=5,
        random_state=42,
        n_jobs=-1
    )),

    # Under-sampling techniques with more parameters
    ('Normalized + RandomUnderSampler', RandomUnderSampler(
        sampling_strategy='auto',
        replacement=False,
        random_state=42
    )),
    ('Normalized + ClusterCentroids', ClusterCentroids(
        sampling_strategy='auto',
        estimator=None,     # Defaults to KMeans
        voting='auto',
        random_state=42
    )),
    ('Normalized + TomekLinks', TomekLinks(
        sampling_strategy='auto'
       
    )),
    ('Normalized + NearMiss (v1)', NearMiss(
        sampling_strategy='auto',
        version=1,
        n_neighbors=5
      
    )),
    ('Normalized + NearMiss (v2)', NearMiss(
        sampling_strategy='auto',
        version=2,
        n_neighbors=5
    )),
    ('Normalized + NearMiss (v3)', NearMiss(
        sampling_strategy='auto',
        version=3,
        n_neighbors_ver3=5
        
    )),

    # Hybrid methods
    ('Normalized + SMOTE + TomekLinks', SMOTETomek(
        sampling_strategy='auto',
        smote=SMOTE(
            sampling_strategy='auto',
            k_neighbors=5,
            random_state=42
        ),
        tomek=TomekLinks(
            sampling_strategy='auto'
        
        ),
        random_state=42,
        n_jobs=-1
    )),
    ('Normalized + SMOTE + ENN', SMOTEENN(
        sampling_strategy='auto',
        smote=SMOTE(
            sampling_strategy='auto',
            k_neighbors=5,
            random_state=42
            
        ),
        enn=None,  # Use default ENN
        random_state=42
        
    )),
]




for cfg_name, sampler in samplers:
    X_res, y_res = sampler.fit_resample(X_train_normalized, y_train)
    configurations.append((cfg_name, X_res, X_test_normalized, y_res))
    print(f"\n=== {cfg_name} ===")
    print("Original X_train:", X_train.shape, "y_train:", y_train.shape)
    print("After SMOTE  X_res  :", X_res.shape, "y_res  :", y_res.shape)
    print("Original class distribution:", dict(Counter(y_train)))
    print("After SMOTE class distribution:", dict(Counter(y_res)))




# Step 2: SVM + GridSearchCV
print("\n=== SVM Model Performance with Hyperparameter Tuning ===")

param_grid = {
    'C': [100],
    'gamma': ['auto'],
    'kernel': ['sigmoid'],
    'degree': [2],
    'coef0': [0.0]
}

for name, X_train_cfg, X_test_cfg, y_train_cfg in configurations:
    print(f"\nRunning SVM with {name} configuration...")
    svc = GridSearchCV(SVC(probability=True), param_grid, cv=2, n_jobs=-1, verbose=2)
    svc.fit(X_train_cfg, y_train_cfg)

    y_train_svc = svc.predict(X_train_cfg)
    y_test_svc = svc.predict(X_test_cfg)
    y_train_svc_proba = svc.predict_proba(X_train_cfg)
    y_test_svc_proba = svc.predict_proba(X_test_cfg)

    metrics_dict = {
        "Dataset": ["Training", "Test"],
        "Accuracy": [
            metrics.accuracy_score(y_train_cfg, y_train_svc),
            metrics.accuracy_score(y_test, y_test_svc),
        ],
        "F1 Score": [
            metrics.f1_score(y_train_cfg, y_train_svc, average='macro'),
            metrics.f1_score(y_test, y_test_svc, average='macro'),
        ],
        "Recall": [
            metrics.recall_score(y_train_cfg, y_train_svc, average='macro'),
            metrics.recall_score(y_test, y_test_svc, average='macro'),
        ],
        "Precision": [
            metrics.precision_score(y_train_cfg, y_train_svc, average='macro'),
            metrics.precision_score(y_test, y_test_svc, average='macro'),
        ],
        "AUC-ROC": [
            metrics.roc_auc_score(pd.get_dummies(y_train_cfg), y_train_svc_proba, multi_class='ovr', average='macro'),
            metrics.roc_auc_score(pd.get_dummies(y_test), y_test_svc_proba, multi_class='ovr', average='macro'),
        ]
    }

    df_metrics = pd.DataFrame(metrics_dict)
    print("\nSupport Vector Machine Model Performance Metrics")
    print(df_metrics.to_string(index=False))

    auc_score = metrics.roc_auc_score(pd.get_dummies(y_test), y_test_svc_proba, multi_class='ovr', average='macro')
    storeResults(
        'Support Vector Machine 99',
        name,
        metrics.accuracy_score(y_test, y_test_svc),
        metrics.f1_score(y_test, y_test_svc, average='macro'),
        metrics.recall_score(y_test, y_test_svc, average='macro'),
        metrics.precision_score(y_test, y_test_svc, average='macro'),
        auc_score
    )

    print("Best hyperparameters found by GridSearchCV:")
    print(svc.best_params_)

results_df = pd.DataFrame({
    "Model": ML_Model,
    "Configuration": ML_Config,
    "Accuracy": accuracy,
    "F1 Score": f1_score,
    "Recall": recall,
    "Precision": precision,
    "AUC-ROC": auc_roc
})

print("\n=== Final Summary of All Results ===")
print(results_df.to_string(index=False))


=== Normalized + RandomOverSampler ===
Original X_train: (280, 49) y_train: (280,)
After SMOTE  X_res  : (492, 49) y_res  : (492,)
Original class distribution: {1: 58, 2: 164, 0: 58}
After SMOTE class distribution: {1: 164, 2: 164, 0: 164}

=== Normalized + SMOTE ===
Original X_train: (280, 49) y_train: (280,)
After SMOTE  X_res  : (492, 49) y_res  : (492,)
Original class distribution: {1: 58, 2: 164, 0: 58}
After SMOTE class distribution: {1: 164, 2: 164, 0: 164}

=== Normalized + Borderline-SMOTE ===
Original X_train: (280, 49) y_train: (280,)
After SMOTE  X_res  : (492, 49) y_res  : (492,)
Original class distribution: {1: 58, 2: 164, 0: 58}
After SMOTE class distribution: {1: 164, 2: 164, 0: 164}

=== Normalized + SVMSMOTE ===
Original X_train: (280, 49) y_train: (280,)
After SMOTE  X_res  : (492, 49) y_res  : (492,)
Original class distribution: {1: 58, 2: 164, 0: 58}
After SMOTE class distribution: {1: 164, 2: 164, 0: 164}

=== Normalized + ADASYN ===
Original X_train: (280, 49) y